Using Pretrained model In this case its *distilbert-base-uncased-emotion* which is a smaller and faster version of BERT. It is fine tuned on emotion classification

In [2]:
! pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Shri_gpt = "bhadresh-savani/distilbert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(Shri_gpt)
print(tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(Shri_gpt)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='bhadresh-savani/distilbert-base-uncased-emotion', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    return inputs

In [5]:
import torch

def predict(text):
    inputs = preprocess(text)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probs


In [6]:
labels = model.config.id2label

def get_emotion_label(probs):
    label_id = torch.argmax(probs, dim=1).item()
    return labels[label_id], probs[0][label_id].item()

text = "Example sentence"
probs = predict(text)
label, confidence = get_emotion_label(probs)
print(f"Emotion: {label}, Confidence: {confidence}")
labels

Emotion: anger, Confidence: 0.48164528608322144


{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

# Testing the model

In [7]:
Examples= [
    "I am very happy today!",
    "I feel so sad and depressed.",
    "I'm scared of what might happen next.",
    "I love spending time with my family.",
    "This is so surprising and unexpected!",
    "I am very angry right now!"
]

for i in Examples:
  probs = predict(i)
  label, confidence = get_emotion_label(probs)
  print(f"Emotion: {label}, Confidence: {confidence}")

Emotion: joy, Confidence: 0.9986829161643982
Emotion: sadness, Confidence: 0.9990469813346863
Emotion: fear, Confidence: 0.9973213076591492
Emotion: joy, Confidence: 0.5013223886489868
Emotion: surprise, Confidence: 0.9423450827598572
Emotion: anger, Confidence: 0.9978970289230347


#Fine Tuning the model

Training model on a different dataset to make it more accurate and usable

In [8]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requi

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import  Trainer, TrainingArguments
import torch

# Load the dataset
file_path = "/content/Untitled spreadsheet - Sheet2.csv"
dataset = pd.read_csv(file_path)

tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

# Encode the sentiments as numeric labels
sentiment_labels = {label: idx for idx, label in enumerate(dataset['sentiment'].unique())}
dataset['labels'] = dataset['sentiment'].map(sentiment_labels)

# Tokenize the content
encodings = tokenizer(dataset['content'].tolist(), truncation=True, padding=True, max_length=128)

# Prepare the dataset for PyTorch
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are long integers
        return item

    def __len__(self):
        return len(self.labels)

# Split the dataset into training and evaluation
train_indices, eval_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)

train_encodings = {key: [val[i] for i in train_indices] for key, val in encodings.items()}
eval_encodings = {key: [val[i] for i in eval_indices] for key, val in encodings.items()}

train_labels = [dataset['labels'].iloc[i] for i in train_indices]
eval_labels = [dataset['labels'].iloc[i] for i in eval_indices]

train_dataset = TweetDataset(train_encodings, train_labels)
eval_dataset = TweetDataset(eval_encodings, eval_labels)


In [15]:
from datasets import load_metric
# Set up the TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

f1_metric = load_metric("f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
    return {
        'f1': f1['f1'],
    }
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-15-7d49c5c929f6>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric = load_metric("f1")


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Step,Training Loss
10,4.262400
20,5.040100
30,4.600400
40,3.387800
50,2.750100
60,2.111600
70,1.897200
80,1.639200
90,1.488800
100,1.582600


TrainOutput(global_step=4173, training_loss=1.0280193615211104, metrics={'train_runtime': 436.8103, 'train_samples_per_second': 76.406, 'train_steps_per_second': 9.553, 'total_flos': 949913337285000.0, 'train_loss': 1.0280193615211104, 'epoch': 3.0})

In [16]:
import numpy as np

evaluation_results = trainer.evaluate()

evaluation_results

{'eval_loss': 1.3414376974105835,
 'eval_f1': 0.5569881296672134,
 'eval_runtime': 9.4209,
 'eval_samples_per_second': 295.3,
 'eval_steps_per_second': 36.939,
 'epoch': 3.0}